In [1]:
import pandas as pd
from os import listdir
from fnmatch import fnmatch
import numpy as np
from astropy.io import fits
from astropy import wcs
from astropy.coordinates import SkyCoord
from astropy import units as u

In [2]:
data = pd.read_csv('data/DFBS.csv')
data.drop(0, inplace=True)
data.head()

,_Glon,_Glat,_RAJ2000,_DEJ2000,Cl,Name,Vmag,z
1,100.174423,-55.203358,0.04875,5.388056,Sy1,RXS J00001+0523,16.4,0.040
2,99.844434,-57.30727,0.61,3.351667,Sy1,MARK 543,14.68,0.026
3,86.112841,-70.112882,0.88375,-10.744722,Sy1,NGC 7808,15.4,0.029
4,114.304767,-16.638006,1.039583,45.440278,Sy1,RXS J00041+4526,16.9,0.120
5,104.972206,-50.897341,1.45625,10.376944,Sy1,RXS J00058+1022,16.7,0.095


In [3]:
headers_path = 'data/fits_headers/'
headers_folder = listdir(headers_path)
headers = []
pattern = "*.hdr"


for entry in headers_folder:
    if fnmatch(entry, pattern):
            headers.append(headers_path + entry)

print(headers[:5], len(headers_folder), len(headers))

headers = np.array(headers)

headers

['data/fits_headers/fbs1164_cor.fits.hdr', 'data/fits_headers/fbs0530_cor.fits.hdr', 'data/fits_headers/fbs0089_cor.fits.hdr', 'data/fits_headers/fbs0057_cor.fits.hdr', 'data/fits_headers/fbs0926_cor.fits.hdr'] 1700 1700


array(['data/fits_headers/fbs1164_cor.fits.hdr',
       'data/fits_headers/fbs0530_cor.fits.hdr',
       'data/fits_headers/fbs0089_cor.fits.hdr', ...,
       'data/fits_headers/fbs0223_cor.fits.hdr',
       'data/fits_headers/fbs0524_cor.fits.hdr',
       'data/fits_headers/fbs1273M_cor.fits.hdr'], dtype='<U41')

In [4]:
data["plate"] = np.nan
data["dx"] = np.zeros(data.shape[0])
data["dy"] = np.zeros(data.shape[0])
data.head()

data['ra'] = data['_RAJ2000'].astype(float)
data['dec'] = data['_DEJ2000'].astype(float)

In [5]:
data.head()

,_Glon,_Glat,_RAJ2000,_DEJ2000,Cl,Name,Vmag,z,plate,dx,dy,ra,dec
1,100.174423,-55.203358,0.04875,5.388056,Sy1,RXS J00001+0523,16.4,0.040,NaN,0.0,0.0,0.048750,5.388056
2,99.844434,-57.30727,0.61,3.351667,Sy1,MARK 543,14.68,0.026,NaN,0.0,0.0,0.610000,3.351667
3,86.112841,-70.112882,0.88375,-10.744722,Sy1,NGC 7808,15.4,0.029,NaN,0.0,0.0,0.883750,-10.744722
4,114.304767,-16.638006,1.039583,45.440278,Sy1,RXS J00041+4526,16.9,0.120,NaN,0.0,0.0,1.039583,45.440278
5,104.972206,-50.897341,1.45625,10.376944,Sy1,RXS J00058+1022,16.7,0.095,NaN,0.0,0.0,1.456250,10.376944


In [6]:
zz = 0
a = np.zeros(len(headers))
plate_rows = []
dxdy = []
coordinates = np.zeros((headers.shape[0], data.shape[0], 2))
counts = np.zeros(data.shape[0])

for i in range(len(headers)):
    hdulist = fits.open(headers[i])
    w = wcs.WCS(hdulist[0].header)
    
    xy = w.all_world2pix(data[['ra', 'dec']], 1, quiet=True)
    
    matching_indices = np.where((xy[:,0] >= 0) & (xy[:,0] <= 9602) & (xy[:,1] >= 0) & (xy[:,1] <= 9602))[0]
    
    coordinates[i][matching_indices] = xy[matching_indices]
    counts[matching_indices] += 1
    
    plate_rows.append(matching_indices)
    dxdy.append(xy[matching_indices])
    
    a[i] += matching_indices.shape[0]

In [7]:
# coord_data = pd.DataFrame(list(coordinates/counts.reshape((-1, 1))), columns=['dx', 'dy'])

In [8]:
coordinates[:,0]

array([[   0.        ,    0.        ],
       [   0.        ,    0.        ],
       [   0.        ,    0.        ],
       ...,
       [   0.        ,    0.        ],
       [5424.19176627,  856.63994337],
       [   0.        ,    0.        ]])

In [9]:
# coord_data.head()

In [10]:
datapoint_plates = [[] for _ in range(len(data))]

for i in range(len(plate_rows)):
    for j in plate_rows[i]:
        datapoint_plates[j].append(i)

In [11]:
datapoint_plates[:5], headers[889]

([[507, 1698], [769], [905, 1021, 1450], [1631], [703]],
 'data/fits_headers/fbs0247_cor.fits.hdr')

In [12]:
np.argsort(a)[-200:-150], a[150],

(array([1277, 1110,  320, 1068,  728,  780,  990,  830,  917,  817,  170,
         495,  175, 1433, 1342,  186,  652,  560,  711, 1398, 1211,  773,
         460,  761,  226, 1368, 1063,  242, 1031,  639, 1215, 1188,  364,
         706, 1153,  389,  393, 1128, 1123,   48,   62,   59, 1677, 1047,
         471,   17, 1615,  370, 1221,   52]),
 14.0)

In [13]:
command1 = 'scp stepan@93.187.165.59:/var/gavo/inputs/dfbs/data/'
command2 = ' /home/stepan/PycharmProjects/DFBS-Object-Classification/data/fits_files'

In [26]:
new_plates = set(map(lambda x: x.split('/')[-1].split('.hdr')[0], np.sort(headers[np.argsort(a)[-383:-280]])))

In [27]:
old_plates = set(listdir('data/fits_files'))

In [30]:
len(new_plates.difference(old_plates))

1

In [35]:
for plate in new_plates.difference(old_plates):
    print(command1 + plate + command2)

scp stepan@93.187.165.59:/var/gavo/inputs/dfbs/data/fbs0482_cor.fits /home/stepan/PycharmProjects/DFBS-Object-Classification/data/fits_files


In [36]:
headers[1036], plate_rows[1036], dxdy[1036][:5]

('data/fits_headers/fbs0631_cor.fits.hdr',
 array([ 582, 1481, 2905, 2907, 2915, 3510]),
 array([[ 569.05721865, 2050.30738012],
        [1122.10394074, 7055.82897565],
        [6909.39165829,  760.30160959],
        [3242.7103123 , 1202.38472103],
        [6093.39310636, 5558.81251384]]))

In [101]:
# np.array(dxdy)
plate_rows

[array([ 564,  570, 1681, 1686, 1687, 2706, 2739, 2749, 2751, 2765, 3485,
        5358], dtype=int64),
 array([ 588,  589, 2711, 2735, 2755, 2771, 5285, 5321], dtype=int64),
 array([2709, 2735, 3101, 3180, 3423, 3887, 5247, 5285], dtype=int64),
 array([ 602, 1552, 2772, 2807, 3597, 4228], dtype=int64),
 array([2746, 2747, 2752, 2764, 2769, 2772, 3636, 3666, 3790, 3887, 4191,
        4336, 4359, 4442, 4532, 4675, 5196, 5213], dtype=int64),
 array([ 667,  668, 1365, 2721, 3320, 3355, 3489, 4506, 5044], dtype=int64),
 array([ 564, 1647, 2790, 6590], dtype=int64),
 array([ 580, 1643, 2780, 2792, 3038, 3859, 4054, 4350, 5275, 5279, 6579,
        6585, 6617], dtype=int64),
 array([1137, 3494, 4263, 4368, 4914, 6946], dtype=int64),
 array([1147, 1155, 1156, 2910, 2912, 2921, 3045, 4131, 6945], dtype=int64),
 array([ 675, 1384, 2797, 3059, 3160, 3165, 4094, 6935], dtype=int64),
 array([3572, 4527, 4991, 4992], dtype=int64),
 array([], dtype=int64),
 array([ 151, 2781, 3964, 4906], dtype=int64)

In [99]:
data.iloc[754]
data.iloc[0]

_Glon            100.174423
_Glat            -55.203358
_RAJ2000            0.04875
_DEJ2000           5.388056
Cl                      Sy1
Name        RXS J00001+0523
Vmag                   16.4
z                      0.04
plate                   NaN
dx                      0.0
dy                      0.0
ra                  0.04875
dec                5.388056
Name: 1, dtype: object

In [179]:
headers[1036].split('/')[-1].split('_')[0]

'fbs1098'

In [113]:
def isNaN(x):
    return str(x) == str(1e400*0)

a = np.zeros(10000)
count = 0
zz = 0

for index, row in data.iterrows():
    lon = float(row["_Glon"])
    lat = float(row["_Glat"])
    ra = row['ra']
    dec = row['dec']

    for i in range(len(headers)):
        hdulist = fits.open(headers[i])
        # print(hdulist)
        w = wcs.WCS(hdulist[0].header)
        # print(w)
        c = SkyCoord(lon, lat, unit=(u.hourangle, u.deg))
        dx, dy = w.wcs_world2pix(ra, dec, 1)
        # print(dx, dy)
        if not(np.isnan(dx)) and not(np.isnan(dy)) and dx >= 0 and dy >= 0 and dx < 10000 and dy < 10000:
            print(dx, dy)
            print('Datapoint:', index)
            print('Header:', i)
            a[i] += 1
            count += 1
        #else:
        #    print("no")
    zz += 1
    if zz == 50:
        break

In [85]:
def check_Valid(array):
    if not(np.isnan(array[0])) and not(np.isnan(array[1])) and array[0] >= 0 and array[1] >= 0 and array[0] < 10000 and array[1] < 10000:
        return True
    return False